In [1]:
import torch
from networks import FullyConnected, SPU, Normalization
import os
import time
import numpy as np
from itertools import product
from box import BoxVerifier
from deeppoly import DeepPolyVerifier

DEVICE = 'cpu'
INPUT_SIZE = 28


class Args:
    net: str
    spec: str

    def __init__(self, net: str, spec: str):
        self.net = net
        self.spec = spec


def analyze(net: torch.nn.Module, inputs: torch.FloatTensor, eps: float, true_label: int) -> str:
    """Returns "verified" if the returned label is the same for all points in a L-infinite epsilon-ball around an input point, and "not verified" otherwise.

    Args:
        net (torch.nn.Module): [description]
        inputs (torch.FloatTensor): [description]
        eps (float): [description]
        true_label (int): [description]

    Returns:
        [str]: Returns "verified" or "not verified"
    """
    verbose = False


    print(f"Network: {net}")


    verifier = DeepPolyVerifier(net=net, inputs=inputs,
                           eps=eps, true_label=true_label, verbose=verbose)
    
    verifier.verify()


args = Args(
    net="net0_fc1",
    spec="../test_cases/net0_fc1/example_img0_0.01800.txt"
)

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])

if args.net.endswith('fc1'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [50, 10]).to(DEVICE)

elif args.net.endswith('fc2'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc3'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 10]).to(DEVICE)

elif args.net.endswith('fc4'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc5'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [
        100, 100, 100, 100, 10]).to(DEVICE)
else:
    assert False

net.load_state_dict(torch.load('../mnist_nets/%s.pt' %
                    args.net, map_location=torch.device(DEVICE)))

inputs = torch.FloatTensor(pixel_values).view(
    1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

outs = net(inputs)
pred_label = outs.max(dim=1)[1].item()
assert pred_label == true_label


if analyze(net, inputs, eps, true_label):
    print('verified')
else:
    print('not verified')


Network: FullyConnected(
  (layers): Sequential(
    (0): Normalization()
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=784, out_features=50, bias=True)
    (3): SPU()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)
Layer 4/5: Linear(in_features=50, out_features=10, bias=True)
Transformer: DeepPolyTransformer(layer: Linear(in_features=50, out_features=10, bias=True))
	Weights shape: (10, 50)
	Bounds shape: (10,)
Previous transformer: DeepPolyTransformer(layer: SPU())
	Weights shape: (50, 50)
	Bounds shape: (50,)
Lower_weights.T shape: (50, 50)
Product shape: (50, 50)
Bias shape: (50,)
Lower shape: (50,)
Upper shape: (50,)
Layer 3/5: SPU()
Transformer: DeepPolyTransformer(layer: None)
	Weights shape: (50, 50)
	Bounds shape: (50,)
Previous transformer: DeepPolyTransformer(layer: Linear(in_features=784, out_features=50, bias=True))
	Weights shape: (50, 784)
	Bounds shape: (50,)
Lower_weights.T shape: (784, 50)
Product shape: (784, 50)
Bias shape: 

ValueError: operands could not be broadcast together with shapes (1,1,28,28) (784,) 

# TESTS

In [5]:
layer_shape = (1,1,4,4)

In [6]:
np.zeros(2 * layer_shape).shape

(1, 1, 4, 4, 1, 1, 4, 4)

In [7]:
tuple(list(map(lambda x: list(x), zip(
    *product(*map(range, layer_shape))))) * 2)


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])

In [13]:
indices = tuple(list(map(lambda x: list(x), zip(*product(*map(range, layer_shape))))))

In [14]:
a = np.arange(16).reshape((4,4))
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [17]:
a[::-1]

array([[12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])

In [3]:
a = [1,2,3,4]
a[:-1][::-1]

[3, 2, 1]

In [4]:
reversed(enumerate(a))

TypeError: 'enumerate' object is not reversible

In [7]:
a = np.array([1,2,3,4,5])
print(a.T.shape)
a.reshape((-1, 1)).shape

(5,)


(5, 1)

In [1]:
a = np.ones((5, 2))
b = np.array([5, 5, 5, 5, 5])

print(f"a: {a}")
print(f"Shape of a: {a.shape}")
print(f"Shape of b: {b.shape}")
print(f"Shape of a + b: {(a + b.reshape(-1,1)).shape}")
print(f"a + b: {a + b.reshape(-1,1)}")

NameError: name 'np' is not defined

In [7]:
a = np.zeros((1,1,5, 5))
np.zeros((25,)).reshape((1,1,5,5))


array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]])